In [90]:
import sys
sys.path.append('../_amigocloud')

from amigocloud import AmigoCloud
import pandas as pd
import geopandas as gpd
from datetime import datetime, timedelta
import collections

from docxtpl import DocxTemplate
from docx.shared import Mm
import docxtpl
import requests
import os

# ID PROYECTO
proyecto_id = 35567
# ID FUNCIONES
buscar_reg_nuevos = 6705

In [91]:
# ejecuta cualquier query sql en el proyecto que se le indique
# requiere el id de proyecto, query a ejecutar y tipo solicitud (get o post)
def ejecutar_query_sql(id_project, query, tipo_sql):
    # define la url del proyecto para ejecutar el querry
    url_proyecto_sql = f'https://app.amigocloud.com/api/v1/projects/{id_project}/sql'
    # crea la estructura de query para amigocloud
    query_sql = {'query': query}
    # variable para almacenar resultado
    resultado_get = ''
    # eleige que tipo de solicitud se realizara (get o post)
    if tipo_sql == 'get': 
        resultado_get = amigocloud.get(url_proyecto_sql, query_sql)
    elif tipo_sql == 'post':
        resultado_get = amigocloud.post(url_proyecto_sql, query_sql)
    else:
        resultado_get = 'Se a seleccionado un tipo de solicitud erroneo.'
    return resultado_get

# ejecuta un query que esta almacenado en un proyecto de amigocloud (generalmente un update),
# requiere id de proyecto e id de query
# retorna cuantas filas fueron afectadas
def ejecutar_query_por_id(id_project, id_query, tipo_sql):
    # obtiene el query basado en el id_project y el id_query
    get_query = amigocloud.get(f'https://app.amigocloud.com/api/v1/projects/{id_project}/queries/{id_query}')
    # se extrae solo el texto del query
    query = get_query['query']
    # ejecuta el query_sql con metodo post y guarda la respuesta
    respuesta_post = ejecutar_query_sql(id_project, query, tipo_sql)
    # retorna el numero de filas afectadas por el query
    return respuesta_post

# convierte de formato YYYY-mm-dd H:M:S+z a d/m/YYYY
def convertir_formato_fecha(fecha):
    new_formato = datetime.strptime(fecha, "%Y-%m-%d %H:%M:%S%z").strftime("%d/%m/%Y")
    return new_formato

In [92]:
def get_unidad_local():
    try:
        # ruta del archivo que contiene la información necesaria
        with open('../../unidad_local.txt', 'r') as file:
            unidad = file.readline().strip()
            return unidad
    except FileNotFoundError:
        print("El archivo 'unidad_local.txt' no se encuentra en la ruta especificada.")
    return None

def get_ruta_local():
    try:
        with open('../ruta_local.txt', 'r') as file:
            ruta = file.readline().strip()
            return ruta
    except FileNotFoundError:
        print("El archivo 'ruta_local.txt' no se encuentra en la ruta especificada.")
    return None

# RUTA ONEDRIVE
unidad = get_unidad_local()
ruta = get_ruta_local()
ruta_completa = os.path.join(unidad, ruta)
ruta_completa

'C:\\Documents\\OneDrive - Ingenio Azucarero Guabira S.A\\_DATOS_PYTHON'

In [93]:
# leer token
acceso = open(ruta_completa + '/_keys/api_amigocloud.key','r')
api_token = acceso.readlines()[0]
acceso.close()
amigocloud = AmigoCloud(token=api_token)
amigocloud

In [101]:
def crear_lista_imagenes(id_proyecto, source_amigo_id, doc):
    print('buscando fotos')
    query_fotos = {'query': 'select * from gallery_67001 where source_amigo_id=\'{amigo_id}\''.format(amigo_id=source_amigo_id)}
    print(query_fotos)
    project = 'https://app.amigocloud.com/api/v1/projects/' + str(id_proyecto) + '/sql'
    select_fotos = amigocloud.get(project, query_fotos)
    fotos = select_fotos['data']
    lista_fotos_inline = []
    print('descargando fotos')
    for foto in fotos:
        print('descargando una foto')
        response = requests.get("https://www-amigocloud.s3.amazonaws.com/gallery/" + foto['s3_filename'])
        file = open(ruta_completa + '/fotos/' + foto['s3_filename'], "wb")
        file.write(response.content)
        file.close()
        lista_fotos_inline.append({'foto': docxtpl.InlineImage(doc, image_descriptor=ruta_completa +'/fotos/' + foto['s3_filename'], width=Mm(60))})
    return lista_fotos_inline

In [150]:
def get_firma(tecnico):
    if tecnico == 'ROGELIO':
        nom = 'ROGELIO ACUÑA'
    elif tecnico == 'JUAN PABLO ':
        nom = 'JUAN PABLO JALDIN'
    elif tecnico == 'MILTON':
        nom = 'MILTON ARANIBAR'
    elif tecnico == 'JORGE ANDRES':
        nom = 'JORGE ANDRES LOPEZ'
    else:
        nom = 'firma_jaldin'
    path_img_firma = ruta_completa + f'/templates/firmas/{nom}.png'
    return path_img_firma

In [148]:
get_firma('dd')

'C:\\Documents\\OneDrive - Ingenio Azucarero Guabira S.A\\_DATOS_PYTHON/templates/firmas/firma_jaldin.png'

In [131]:
def generar_reporte(cod):
    id = cod
    query = {'query': 'select mu.*, (select count(*) from dataset_365891 da where mu.amigo_id = da.muestra_ref_datos) from dataset_365887 mu where id={idd}'.format(idd=id)}
    select = amigocloud.get('https://app.amigocloud.com/api/v1/projects/35567/sql', query)
    data = select['data'][0]
    data['fecha_registro'] = datetime.strptime(data['fecha_registro'].split(' ')[0], '%Y-%m-%d').strftime('%d/%m/%Y')
    print(data)
    insp = {
        'fecha':data['fecha_registro'],
        'canhero':data['canhero'],
        'propiedad':data['propiedad'].split('--')[0],
        'lote':data['lote'],
        'area':data['area'],
        'variedad':data['variedad'],
        'edad':data['edad'],
        'ciclo':data['ciclo'],
        'num_mus':data['count']*20,
        'id':data['id'],
        'amigo_id':data['amigo_id']
    }
    insp = collections.namedtuple("insp", insp.keys())(*insp.values())  

    query = {'query': 'select * from dataset_365891 where muestra_ref_datos=\'{amigo_id}\''.format(amigo_id=data['amigo_id'])}
    select = amigocloud.get('https://app.amigocloud.com/api/v1/projects/35567/sql', query)
    datos = select['data']
    tabla_datos = pd.DataFrame(datos)[['id', 'total_yemas', 'perforadas', 'enraizamiento', 'brotacion', 'muerta']]

    total_yemas = tabla_datos['total_yemas'].sum()
    total_perforadas = tabla_datos['perforadas'].sum()
    total_enraizamiento = tabla_datos['enraizamiento'].sum()
    total_brotacion = tabla_datos['brotacion'].sum()
    total_muertas = tabla_datos['muerta'].sum()
    total_inviables = total_perforadas + total_enraizamiento + total_brotacion + total_muertas
    total_viables = total_yemas - total_inviables

    porcen_perforadas = (total_perforadas / total_yemas)*100
    porcen_enraizamiento = (total_enraizamiento / total_yemas)*100
    porcen_brotacion = (total_brotacion / total_yemas)*100
    porcen_muertas = (total_muertas / total_yemas)*100
    porcen_inviables = (total_inviables / total_yemas)*100
    porcen_viables = (total_viables / total_yemas)*100

    res = {
        'diat':total_perforadas,
        'en':total_enraizamiento,
        'bro':total_brotacion,
        'mur':total_muertas,
        'invi':total_inviables,
        'via':total_viables,
        'ana':total_yemas,
        'diatp':porcen_perforadas,
        'enp':porcen_enraizamiento,
        'brop':porcen_brotacion,
        'murp':porcen_muertas,
        'invip':porcen_inviables,
        'viap':porcen_viables
    }
    res = collections.namedtuple("res", res.keys())(*res.values())
    
    doc = DocxTemplate(ruta_completa + "/templates/tpl_rpt_calidad_semilla.docx")
    id_proyecto = '35567'
    source_amigo_id = insp.amigo_id
    inline_fotos = crear_lista_imagenes(proyecto_id, source_amigo_id, doc)
    context = {'insp':insp, 'res':res, 'fotos':inline_fotos }
    doc.render(context)
    ca = insp.canhero.split(' / ')
    file_name = ca[0] + '_CDSE_' + insp.fecha.replace('/', '-') + '_' + ca[1] + '_' + str(insp.id) + '_' + str(insp.lote)
    doc.save(ruta_completa + '/informes/_' + file_name + '.docx')

In [136]:
def generar_reportes(df):
    for i, row in df.iterrows():
        insp = {
            'id':row['id'],
            'fecha': convertir_formato_fecha(row['fecha']),
            'tecnico':row['tecnico'],
            'canhero':row['canhero'],
            'propiedad':row['propiedad'].split('--')[0],
            'lote':row['lote'],
            'variedad':row['variedad'],
            'ciclo':row['ciclo'],
            'edad':row['edad'],
            'area':row['area'],
            'obs':row['obs'],
            'amigo_id':row['amigo_id'],
            't_yemas':row['total_yemas'],
            't_perf':row['total_perf'],
            'p_perf':row['total_perf'] / row['total_yemas'] * 100,
            't_enrai':row['total_enrai'],
            'p_enrai':row['total_enrai'] / row['total_yemas'] * 100,
            't_brota':row['total_brota'],
            'p_brota':row['total_brota'] / row['total_yemas'] * 100,
            't_muer':row['total_muer'],
            'p_muer':row['total_muer'] / row['total_yemas'] * 100,
            't_inviables':row['total_inviables'],
            'p_inviables':row['porcen_inviables'] * 100,
            't_viables':row['total_viables'],
            'p_viables':row['porcen_viables'] * 100,
            'estado':row['estado'], 
        }
        insp = collections.namedtuple("insp", insp.keys())(*insp.values()) 
        doc = DocxTemplate(ruta_completa + "/templates/tpl_rpt_calidad_semilla.docx")
        source_amigo_id = insp.amigo_id
        inline_fotos = crear_lista_imagenes(proyecto_id, source_amigo_id, doc)
        inline_firma = docxtpl.InlineImage(doc, image_descriptor=get_firma(insp.tecnico), width=Mm(60))
        context = {'insp':insp, 'firma':inline_firma, 'fotos':inline_fotos }
        doc.render(context)
        ca = insp.canhero.split(' / ')
        file_name = ca[0] + '_CDSE_' + insp.fecha.replace('/', '-') + '_' + ca[1] + '_' + str(insp.id)
        doc.save(ruta_completa + '/informes/_' + file_name + '.docx')

In [133]:
new_registros = ejecutar_query_por_id(proyecto_id, buscar_reg_nuevos, 'get')
df_new_data = pd.DataFrame(new_registros['data'])
df_new_data

,id,fecha,tecnico,canhero,propiedad,lote,variedad,ciclo,edad,area,...,total_yemas,total_perf,total_enrai,total_brota,total_muer,total_inviables,porcen_inviables,total_viables,porcen_viables,estado
0,200,2025-03-31 19:16:06+00:00,JUAN PABLO,1215 / AGUILERA RIVERO OLGA KARINA,1892 / SANTA ROSITA--SERRATE HUGO,L2,UCG 90-20,SOCA 1,9,2.8,...,300.0,15.0,6.0,0.0,0.0,21.0,0.07,279.0,0.93,EXCELENTE
1,204,2025-04-01 02:26:22+00:00,ROGELIO,10380 / PAICHUCAMA CACERES JOSE FELIX,None,None,UCG 90-20,SOCA 1,10,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,TERRIBLE


In [134]:
df_validos = df_new_data[df_new_data['propiedad'].notna() & df_new_data['total_yemas'].notna()]
df_invalidos = df_new_data[df_new_data['propiedad'].isna() | df_new_data['total_yemas'].isna()]

In [151]:
if len(df_validos) > 0:
    generar_reportes(df_validos)
else:
    print('No existen registros validos nuevos')
if len(df_invalidos) > 0:
    print('Existen registros con datos no validos')

buscando fotos
{'query': "select * from gallery_67001 where source_amigo_id='da5ebacbd6c74d3ba01b907548a19c23'"}
descargando fotos
descargando una foto
Existen registros con datos no validos


In [ ]:
fecha_insp = '2025-02-18'
query = {'query': 'select id, canhero, propiedad, lote, variedad from dataset_365887 where fecha_registro::date=\'{fecha}\''.format(fecha=fecha_insp)}
select = amigocloud.get('https://app.amigocloud.com/api/v1/projects/35567/sql', query)
data = select['data']
tabla = pd.DataFrame(data)
tabla

In [14]:
fecha_insp = '2025-02-18'
query = {'query': 'select id, canhero, propiedad, lote, variedad from dataset_365887 where fecha_registro::date=\'{fecha}\''.format(fecha=fecha_insp)}
select = amigocloud.get('https://app.amigocloud.com/api/v1/projects/35567/sql', query)
data = select['data']
tabla = pd.DataFrame(data)
tabla

,id,canhero,propiedad,lote,variedad
0,130,13067 / SAUCEDO CERRUTI SONIA,2076 / NAICO--SAUCEDO SONIA,B12,UCG 90-20
1,131,18086 / SUAREZ CUELLAR JENNY SONIA,550 / LA IMPROVISADA--SUAREZ,L12,CITTCA 0563


In [15]:
ids = list(tabla['id'])
ids

[130, 131]

In [16]:
for i in ids:
    generar_reporte(i)
    print(i)

{'fecha_registro': '18/02/2025', 'id': 130, 'estado': 'APROBADO', 'ubicacion': '0101000020E61000007F5F7EDB28A74FC08A37F3161F4031C0', 'tecnico_responsable': 'JUAN PABLO', 'canhero': '13067 / SAUCEDO CERRUTI SONIA', 'lote': 'B12', 'observaciones': '', 'propiedad': '2076 / NAICO--SAUCEDO SONIA', 'variedad': 'UCG 90-20', 'ciclo': 'HOJA', 'area': 4.8, 'edad': 8, 'usuario': 'Juan Pablo Jaldin Duran', 'amigo_id': '50A2AFB7A10B486C9D542B4EC8EEFCF7', 'procesado': True, 'count': 1}
buscando fotos
{'query': "select * from gallery_67001 where source_amigo_id='50A2AFB7A10B486C9D542B4EC8EEFCF7'"}
descargando fotos
130
{'fecha_registro': '18/02/2025', 'id': 131, 'estado': 'APROBADO', 'ubicacion': '0101000020E61000008A2DF30CD0984FC0CB08B99C5C4F31C0', 'tecnico_responsable': 'MILTON', 'canhero': '18086 / SUAREZ CUELLAR JENNY SONIA', 'lote': 'L12', 'observaciones': None, 'propiedad': '550 / LA IMPROVISADA--SUAREZ', 'variedad': 'CITTCA 0563', 'ciclo': 'SOCA 1', 'area': 13.4, 'edad': 10, 'usuario': '', 'am